# 4. The Standard Model

In this chapter, we'll go through all fields in the standard model, explain their meaning and the default functions that calculate them.

In [ ]:
from tripod import Simulation
import numpy as np

In [ ]:
sim = Simulation()

In [ ]:
sim.ini.grid.Nr = 20 
sim.makegrids()
sim.initialize()

By default, the frame object of `TwoPopPy` consists of four groups for **dust**, **gas**, **grid**, and **stellar** parameters, a field for the **time**, which is the integration variable, an **integrator** object, and a **writer** object.

In [ ]:
sim

## Dust

In [ ]:
sim.dust

### `Simulation.dust.backreaction`

In [ ]:
sim.dust.backreaction

The backreaction describes the hydrodynamic influence the dust has on the gas. Numerically, it consists of two fields `A` and `B` of shape `(Simulation.grid.Nr,)` that describe the pull or rather the push the dust excerts on the gas respectively.

The details of this mechanism are described in [Gárate et al. (2019)](https://doi.org/10.3847/1538-4357/aaf4fc).

Backreaction modifies the radial gas velocity as follows

$v_\mathrm{g} = Av_\mathrm{visc} + 2B\eta v_\mathrm{K}$.

In the standard model, we have `A=1` and `B=0` everywhere, i.e. backreactions are not active.

In [ ]:
sim.dust.backreaction.A

In [ ]:
sim.dust.backreaction.B

Depending on the type of backreaction that we want to model, we have to provide functions for `A` and `B`. Please also have a look at Appendix A of [Gárate et al. (2019)](https://doi.org/10.3847/1538-4357/aaf4fc) for examples.

### `Simulation.dust.boundary`

In [ ]:
sim.dust.boundary

By default, the inner dust boundary is set to constant gradient and the outer boundary to floor value.

In [ ]:
sim.dust.boundary.inner

In [ ]:
sim.dust.boundary.outer

The outer boundary therefore stores an array with the floor values of all particle masses.

The boundary conditions can be changed via `setcondition()`.

In [ ]:
help(sim.dust.boundary.outer.setcondition)
sim.dust.boundary.inner.setcondition("grad", -1)

### `Simulation.dust.delta`

In [ ]:
sim.dust.delta

The $\delta$ parameters control the mixing of dust particles along vertical and radial directions and turbulent mixing. We can look at them as similar to the turbulent $\alpha$ parameter for the gas. By default, they will also have the same value as $\alpha$ as given by `Simulation.ini.gas.alpha`.

#### `Simulation.dust.delta.rad`

In [ ]:
sim.dust.delta.rad

$\delta_\mathrm{rad}$ will be used to calculate the radial dust diffusion.

#### `Simulation.dust.delta.turb`

In [ ]:
sim.dust.delta.turb

$\delta_\mathrm{turb}$ will be used to calculate the turbulent collision velocities of the dust particles.

#### `Simulation.dust.delta.vert`

In [ ]:
sim.dust.delta.vert

$\delta_\mathrm{vert}$ will be used to calculate the vertical dust scale heights.

### `Simulation.dust.Fi`

In [ ]:
sim.dust.Fi

This is a group of fluxes through the radial grid interfaces for every particle mass of shape `(Simulation.grid.Nr+1, 2)`.

**Attention:** When using implicit dust integration, the interface fluxes are calculated in retrospect after the new dust surface density was found. Changing any `Simulation.dust.Fi` does not influence the simulation. It is only given for data analysis.

#### `Simulation.dust.Fi.adv`

This is the advective flux calculated by $F_\mathrm{adv} = v_\mathrm{d}\Sigma_\mathrm{d}$.

**Attention:** When using implicit dust integration, the advective interface fluxes are calculated in retrospect after the new dust surface density was found. Changing `Simulation.dust.Fi.adv` does not influence the simulation. It is only given for data analysis.

#### `Simulation.dust.Fi.diff`

This is the diffusive flux calculated by $F_\mathrm{diff} = -D\Sigma_\mathrm{g}\nabla\frac{\Sigma_\mathrm{d}}{\Sigma_\mathrm{gas}}$ for every particle species separately. The diffusive fluxes at the grid boundaries are set to zero to avoid instabilities.

**Attention:** When using implicit dust integration, the diffusive interface fluxes are calculated in retrospect after the new dust surface density was found. Changing `Simulation.dust.Fi.diff` does not influence the simulation. It is only given for data analysis.

#### `Simulation.dust.Fi.tot`

This is the total flux through the radial grid interfaces $F_\mathrm{tot} = F_\mathrm{adv} + F_\mathrm{diff}$.

**Attention:** When using implicit dust integration, the total interface fluxes are calculated in retrospect after the new dust surface density was found. Changing `Simulation.dust.Fi.tot` does not influence the simulation. It is only given for data analysis.

### `Simulation.dust.p`

In [ ]:
sim.dust.p

These are the fragmentation and sticking probability of certain particle collisions.  
Each cell has a scalar probability, e.g the shape of p.frag is `(Simulation.grid.Nr)`.

Collisions between particles of sizes $0.4*a_\text{max}$ and $a_\text{max}$ are representative for the entire dust distribution. The characteristic fragmentation probability at radial grid cell `ir=15` is therefore given by

In [ ]:
ir = 15
sim.dust.p.frag[ir]


#### `Simulation.dust.p.frag`

`TriPoD` assumes that the relative collision velocities follow the Maxwell-Boltzmann distribution

$f\left(\Delta v\right) = \sqrt{\frac{54}{\pi}}\frac{\Delta v^{2}}{v_\mathrm{rms}^3} \exp \left[-\frac{3}{2} \frac{\Delta v^2}{v_\mathrm{rms}^2} \right]$.

The collision kernel for fragmentation is then given by

$K_\mathrm{f} = \int\limits_{v_\mathrm{frag}}^\infty \sigma_\mathrm{geo} \Delta v f\left(\Delta v\right) \mathrm{d}\Delta v$

by counting all collisions above the fragmentation velocity.

The fragmentation probability is then given by

$p_\mathrm{f} = \frac{K_\mathrm{f}}{\sigma_\mathrm{geo} \Delta \bar{v}}$

with the mean velocity of the Maxwell-Boltzmann distribution

$\Delta \bar{v} = \sqrt{\frac{8\pi}{2}}v_\mathrm{rms}$.

The root-mean-square velocities $v_\mathrm{rms}$ are stored in `Simulation.dust.v.rel`.

#### `Simulation.dust.p.stick`

This is the sticking probability given by $p_\mathrm{s} = 1 - p_\mathrm{f}$.

No bouncing include yet.

### `Simulation.dust.S`

In [ ]:
sim.dust.S

These are the source terms of the dust of shape `(Simulation.grid.Nr, 2)` used to integrate the time evolution of the dust.

#### `Simulation.dust.S.coag`

The source terms from dust coagulation given by 

$S_{\mathrm{coag},\,0 \rightarrow 1} = \dot{\Sigma}_{0 \rightarrow 1} - \dot{\Sigma}_{1 \rightarrow 0} =  \frac{\Sigma_0 \Sigma_1 \sigma_{01} \Delta v_{01}}{m_1 \sqrt{2 \pi \left(H_0^2 + H_1^2\right)}} - \frac{\Sigma_1^2 \sigma_{11} \Delta v_{11}}{m_1 2 \sqrt{\pi} H_1} \mathcal{F} = - S_{\mathrm{coag},\,1 \rightarrow 0}$

with $\mathcal{F} = \sqrt{\frac{2H_1^2}{H_0^2 + H_1^2}} \frac{\sigma_{01}\Delta v_{01}}{\sigma_{11}\Delta v_{11}}\left(\frac{s_{max}}{s_{min}}\right)^{-\left(\xi'+4\right)}$ and $\xi' = \Pi_{frag}\xi_{frag} + \Pi_{stick}\xi_{stick}$

#### `Simulation.dust.S.ext`

External source terms. These are by default set to zero, i.e. no external sources.

In [ ]:
sim.dust.S.ext

#### `Simulation.dust.S.hyd`

These are the hydrodynamic source terms. At grid cell $i$ they are calculated by

$S_{\mathrm{hyd},\,i} = 2\frac{\left( r_{i-\frac{1}{2}}F_{i-\frac{1}{2}}-r_{i+\frac{1}{2}}F_{i+\frac{1}{2}} \right)}{r_{i+\frac{1}{2}}^2 - r_{i-\frac{1}{2}}^2}$.

**Attention:** When using implicit dust integration, the hydrodynamic source terms are calculated in retrospect after the new dust surface density was found. Changing `Simulation.dust.S.hyd` does not influence the simulation. It is only given for data analysis.

#### `Simulation.dust.S.tot`

These are the total source terms given by $S_\mathrm{tot} = S_\mathrm{coag} + S_\mathrm{ext} + S_\mathrm{hyd}$.

### `Simulation.dust.s`

In [ ]:
sim.dust.s

These are specific particle sizes of shape `(Simulation.grid.Nr)` characterizing the dust distribution.

#### `Simulation.dust.s.max`

This is the maximum particle size of the dust distribution. By default, it is initialized as `Simulation.ini.dust.aIniMax`.

In [ ]:
sim.dust.s.max

Over the course of the simulation, it is evolved in time according to the source term

$\dot{s}_{max} = \frac{\rho_d}{\rho_s}\Delta v \left[\frac{1-\left(\frac{v_{frag}}{\Delta v}\right)^s}{1+\left(\frac{v_{frag}}{\Delta v}\right)^s}\right]$

describing its growth and decline due to sticking and fragmentation.

#### `Simulation.dust.s.min`

This is the minimum particle size of the dust distribution, which remains constant over time. It is by default set to $0.1 *$ `Simulation.ini.dust.aIniMax`.

In [ ]:
sim.dust.s.min

#### `Simulation.dust.s.lim`

 This is the minimum particle size of the dust distribution, which form the gap shrikage term as described in Pfeil+ 2024

In [ ]:
sim.dust.s.lim

### `Simulation.dust.v`

In [ ]:
sim.dust.v

These are some dust related velocities the simulation needs for execution. 

#### `Simulation.dust.v.rel`

In [ ]:
sim.dust.v.rel

These are the different sources of relative particle collision velocities used by the coagulation algorithm.  
They are used for two different reasons:
1. To calculated the collision rates
2. To determine the outcome of a collision

#### `Simulation.dust.v.rel.azi`

Relative collision velocity caused by a differential azimuthal drift of particles with different Stokes numbers calculated by

$v_{\mathrm{rel},\,\mathrm{azi}} = \left| v_{\mathrm{drift},\,\mathrm{max}} \cdot \left( \frac{1}{1+\mathrm{St}_i^2} - \frac{1}{1+\mathrm{St}_j^2} \right) \right|$.

##### `Simulation.dust.v.brown`

Relative collision velocity of particles caused by Brownian motion calculated with

$v_{\mathrm{rel},\,\mathrm{brown}} = \sqrt{ \frac{8k_\mathrm{B}T\left(m_i + m_j \right)}{\pi m_i m_j} }$.

Since this expression is diverging for very small particle masses, the relative velocity is capped to a maximum value of the sound speed $c_\mathrm{s}$. For very small particles this can still be larger than the fragmentation velocity and can cause unwanted fragmentation in the simple coagulation algorithm implemented in the default model.

##### `Simulation.dust.v.rel.rad`

Relative collision velocity caused by differential radial drift.

$v_{\mathrm{rel},\,\mathrm{rad}} = \left| v_{\mathrm{rad},\,i} - v_{\mathrm{rad},\,j} \right|$.

##### `Simulation.dust.v.rel.tot`

Total relative velocities calculated by using the root mean square of all individual velocity sources.

$v_{\mathrm{rel},\,\mathrm{tot}} = \sqrt{v_{\mathrm{rel},\,\mathrm{azi}}^2 + v_{\mathrm{rel},\,\mathrm{brown}}^2 + v_{\mathrm{rel},\,\mathrm{rad}}^2 + v_{\mathrm{rel},\,\mathrm{turb}}^2 + v_{\mathrm{rel},\,\mathrm{vert}}^2}$.

##### `Simulation.dust.v.rel.turb`

Relative velocities caused by turbulent motion of the particles. This follows the prescription of [Ormel & Cuzzi (2007)](https://doi.org/10.1051/0004-6361:20066899).  
It uses `Simulation.dust.delta.turb` instead of `Simulation.gas.alpha` to calculate the velocities.

##### `Simulation.dust.v.rel.vert`

Relative collision velocities caused by differential vertical settling of particles.

$v_{\mathrm{rel},\,\mathrm{vert}} = \left| h_i \min \left( \mathrm{St}_i,\,\frac{1}{2}\right) - h_j \min \left( \mathrm{St}_j,\,\frac{1}{2}\right) \right| \cdot \Omega_\mathrm{K}$.

This prescription is taken from [Birnstiel et al. (2010)](https://doi.org/10.1051/0004-6361/200913731) and follows [Dullemond & Dominik (2004)](https://doi.org/10.1051/0004-6361:20040284).

#### `Simulation.dust.v.driftmax`

This is the maximum drift velocity a particle of $\mathrm{St} = 1$ can have.

$v_{\mathrm{drift},\,\mathrm{max}} = \frac{1}{2} B v_\mathrm{visc} - A \eta v_\mathrm{K}$.

See `Simulation.dust.backreaction` for details.

#### `Simulation.dust.v.frag`

Fragmentation velocities of shape `(Simulation.grid.Nr)`. By default this is set by the value of `Simulation.ini.dust.vfrag`.

In [ ]:
sim.dust.v.frag

#### `Simulation.dust.v.rad`

Radial velocities of the dust particles.

$v_\mathrm{rad} = \left( v_\mathrm{g} + 2 v_{\mathrm{drift},\,\mathrm{max}}\mathrm{St} \right) \frac{1}{\mathrm{St}^2+1}$

#### `Simulation.dust.v.rad_flux`

Radial velocities of the dust particles used for the evolution of the surface densities which includes the callibration factor $f_{drift}$.

$v_\mathrm{rad} = f_{drift} \left( v_\mathrm{g} + 2 v_{\mathrm{drift},\,\mathrm{max}}\mathrm{St} \right) \frac{1}{\mathrm{St}^2+1}$

### `Simulation.dust.qrec`


This is the exponent of the current particle size distribution calculated via

$q = \frac{\log\left(\Sigma_1/\Sigma_0\right)}{\log\left(s_{max}/s_{int}\right)} - 4$

In [ ]:
sim.dust.qrec

These are exponents of the particle size distribution $\sigma\left(a\right) \propto a^\xi$ of shape `(Simulation.grid.Nr)`.

#### `Simulation.dust.q.eff`

This is the exponent of the particle size distribution strives for in the equilibrium

In [ ]:
sim.dust.q.eff

#### `Simulation.dust.q.frag`

This is the exponent of the particle size distribution in the fragmentation limited case. It is initialized as negative `Simulation.ini.distExp` by default.

In [ ]:
sim.dust.q.frag

#### `Simulation.dust.q.sweep`

This is the exponent of the particle size distribution in the drift limited case. By default, it is set to negative `Simulation.ini.distExp` $+ 1$.

In [ ]:
sim.dust.q.sweep

### `Simulation.dust.a`

Particle radii of shape `(Simulation.grid.Nr, 5)` calculated with

$[a_0, 0.4 * a_1, a_1, 0.4 * a_{max}, a_{max}]$

$a_0 = \left\{ \begin{aligned} \frac{s_{int}s_{min}}{s_{int}-s_{min}} \log\left(s_{int}/s_{min}\right) \textrm{ for }\xi = -5 \\ \frac{s_{int}-s_{min}}{\log\left(s_{int}/s_{min}\right)} \textrm{ for }\xi = -4 \\ \frac{\xi+4}{\xi+5} \frac{s_{int}^{\xi+5}-s_{min}^{\xi+5}}{s_{int}^{\xi+4}-s_{min}^{\xi+4}} \textrm{ for }\xi \neq -4, -5\end{aligned} \right.$

$a_1 = \left\{ \begin{aligned} \frac{s_{max}s_{int}}{s_{max}-s_{int}} \log\left(s_{max}/s_{int}\right) \textrm{ for }\xi = -5 \\ \frac{s_{max}-s_{int}}{\log\left(s_{max}/s_{int}\right)} \textrm{ for }\xi = -4 \\ \frac{\xi+4}{\xi+5} \frac{s_{max}^{\xi+5}-s_{int}^{\xi+5}}{s_{max}^{\xi+4}-s_{int}^{\xi+4}} \textrm{ for }\xi \neq -4, -5\end{aligned} \right.$


### `Simulation.dust.D`

Dust diffusivity for every particle species of shape `(Simulation.grid.Nr, 5)` calculated with

$D = \frac{\delta_\mathrm{rad}\,c_\mathrm{s}^2}{\Omega_\mathrm{K}\left( 1 + \mathrm{St}^2 \right)}$.

### `Simulation.dust.eps`

This is the vertically integrated dust-to-gas ratio of shape `(Simulation.grid.Nr)`. In the literature this is also often refered to as metallicity $z$. It is calculated via

$\epsilon = \frac{\sum\limits_i \Sigma_{\mathrm{d},\,i}}{\Sigma_\mathrm{g}}$

### `Simulation.dust.fill`

This describes the filling factor of the dust aggregates. By default this is 1.

<span style="color: red;"> NOTE: do not make this size dependent otherwies the algorithm breaks down

In [ ]:
sim.dust.fill

### `Simulation.dust.H`

These are the dust scale heights of shape `(Simulation.grid.Nr, 5)` calculated with the prescription of [Dubrulle et al. (1995)](https://doi.org/10.1006/icar.1995.1058)

$H_i = H_\mathrm{P} \cdot \sqrt{\frac{\delta_\mathrm{vert}}{\delta_\mathrm{vert}+\mathrm{St}_i}}$

It is limited to a maximum of the pressure scale height $H_\mathrm{P}$. It uses `Simulation.dust.delta.vert` as vertical mixing parameter instead of `Simulation.gas.alpha`.

### `Simulation.dust.m`

Particle masses of shape `(Simulation.grid.Nr, 5)` calculated with

$m = \frac{4}{3} \pi \rho_s a^3 * fill$

### `Simulation.dust.rho`

Midplane mass densities of the dust.

$\rho = \frac{\Sigma}{\sqrt{2\pi}H}$

### `Simulation.dust.rhos`

Solid state density of the particle material. This is initially set to `Simulation.ini.dust.rhoMonomer`.

this will be modified if you add mutiple active dust species -> see next notebook

In [ ]:
sim.dust.rhos

To calculate the mass density of a dust aggregate do

In [ ]:
sim.dust.fill * sim.dust.rhos

This is the density that is used to calculate the particle size.

### `Simulation.dust.Sigma`

This is the dust surface density of the small and large particle species of shape `(Simulation.grid.Nr, 2)`. It is the quantity that is integrated to solve for dust evolution.

The surface densities are integrated over the mass bin, so a numerical summation over the mass dimension returns the total dust surface density.

$\Sigma_\mathrm{d} = \sum\limits_i \Sigma_{\mathrm{d},\,i}$

In [ ]:
sim.dust.Sigma.sum(-1)

If you want to plot the dust density distribution, you might want to convert the field into a quantity that does not depend on the mass bin width. The plotting script integrated in `TwoPopPy` is plotting a grid independent density distribution. See the first chapter for details on this.

### `Simulation.dust.SigmaFloor`

This is the floor value for the dust surface densities. Mass bins that are below their respective floor value will not contribute to coagulation. By default, the floor value is the density that corresponds to one physical particle of that mass distributed over an annulus at that radial grid location.

$\Sigma_{\mathrm{d},\,\mathrm{floor}} = \frac{m}{A_\mathrm{annulus}}$

Densities below the floor value therefore correspond to fewer that one physical particle in that radial grid cell.

### `Simulation.dust.St`

Stokes numbers of the particles. By default the Epstein and the Stokes I drag regime are considered.

$\mathrm{St} =
\begin{cases}
\frac{\pi}{2} \frac{a\rho}{\Sigma_\mathrm{g}}, & \text{if } a < \frac{9}{4} \lambda_\mathrm{mfp}\\
\frac{2\pi}{9} \frac{a^2 \rho}{\lambda_\mathrm{mfp} \Sigma_\mathrm{g}}, & \text{else}
\end{cases}$

### Update order

The update order of the dust quantities in the standard model is set to

In [ ]:
sim.dust.updateorder

In [ ]:
sim.dust.backreaction.updateorder

In [ ]:
sim.dust.delta.updateorder

In [ ]:
sim.dust.Fi.updateorder

In [ ]:
sim.dust.p.updateorder

In [ ]:
sim.dust.S.updateorder

In [ ]:
sim.dust.s.updateorder

In [ ]:
sim.dust.v.updateorder

In [ ]:
sim.dust.v.rel.updateorder

In [ ]:
sim.dust.q.updateorder

**Note**: The quantities that are excluded are calculated in the finalization step of the implicit integrator from the new values of the dust surface densities.

## Gas

In [ ]:
sim.gas

### `Simulation.gas.boundary`

In [ ]:
sim.gas.boundary

These are the boundary conditions of the gas. By default the inner boundary is set to constant gradient and the outer boundary to floor value.

In [ ]:
sim.gas.boundary.inner

In [ ]:
sim.gas.boundary.outer

The boundary conditions can be modified with `setcondition`.

In [ ]:
help(sim.gas.boundary.inner.setcondition)

If the gas surface density follows a power law $\propto R^{-1}$ the constant gradient boundary condition should work fine. Other values can lead to deviations at the inner boundary. See the chapter about gas evolution tests for details.

### `Simulation.gas.S`

In [ ]:
sim.gas.S

These are the source terms of the gas.

#### `Simulation.gas.S.ext`

These are the external source terms for gas evolution, e.g. infall. By default these are set to zero.

In [ ]:
sim.gas.S.ext

#### `Simulation.gas.S.hyd`

These are the hydrodynamic source terms of the gas evolution.  
**Attention:** Since the gas evolution is calculated implicitly, the hydrodynamic source terms are calculated in retrospect after the new gas surface density was found. Changing `Simulation.gas.S.hyd` does not influence the simulation. It is only given for data analysis.

#### `Simulation.gas.S.tot`

These are the total source terms of gas evolution.

$S_\mathrm{tot} = S_\mathrm{ext} + S_\mathrm{hyd}$

**Attention:** Since the gas evolution is calculated implicitly, the total source terms are calculated in retrospect after the new gas surface density was found. Changing `Simulation.gas.S.tot` does not influence the simulation. It is only given for data analysis.

### `Simulation.gas.v`

In [ ]:
sim.gas.v

These are velocities that are relevant for the gas evolution.  
**Attention:** Since the gas evolution is calculated implicitly, the velocities are calculated in retrospect after the new gas surface density was found. Changing anything in `Simulation.gas.v` does not influence the simulation. It is only given for data analysis.

#### `Simulation.gas.v.rad`

This is the radial gas velocity. It is given by

$v_\mathrm{g} = Av_\mathrm{visc} + 2B\eta v_\mathrm{K}$.

See `Simulation.dust.backreaction` for details. If backreaction is turned off, i.e., $A=1$ and $B=0$, the radial velocity is identical to the viscous velocity.  
**Attention:** Since the gas evolution is calculated implicitly, the velocities are calculated in retrospect after the new gas surface density was found. Changing anything in `Simulation.gas.v.rad` does not influence the simulation. It is only given for data analysis.

#### `Simulation.gas.v.visc`

This is the radial viscous gas velocity

$v_\mathrm{visc} = -\frac{3}{\Sigma_\mathrm{g}\sqrt{R}} \frac{\partial}{\partial R} \left( \Sigma_\mathrm{g} \nu \sqrt{R} \right)$

**Attention:** Since the gas evolution is calculated implicitly, the velocities are calculated in retrospect after the new gas surface density was found. Changing anything in `Simulation.gas.v.visc` does not influence the simulation. It is only given for data analysis.

### `Simulation.gas.alpha`

This is the turbulent viscosity parameter according [Shakura & Sunyaev (1973)](https://ui.adsabs.harvard.edu/abs/1973A%26A....24..337S/abstract). It is initially set to the value in `Simulation.ini.gas.alpha`.

In [ ]:
sim.gas.alpha

### `Simulation.gas.cs`

This is the adiabatic sound speed in the midplane of the disk

$c_\mathrm{s} = \sqrt{\frac{k_\mathrm{B} T}{\mu}}$.


### `Simulation.gas.eta`

This is the midplane pressure gradient parameter $\eta$ given by

$\eta = -\frac{1}{2} \left( \frac{H_\mathrm{P}}{r} \right)^2 \frac{\partial \log P}{\partial \log r}$

It describes the degree of "sub-Keplerity" of the disk

$v_\phi^2 = \left( 1-2\eta \right) v_\mathrm{K}^2$.

### `Simulation.gas.Fi`

These are the mass fluxes of gas through the grid cell interfaces.  
**Attention:** Since the gas evolution is calculated implicitly, the fluxes are calculated in retrospect after the new gas surface density was found. Changing anything in `Simulation.gas.Fi` does not influence the simulation. It is only given for data analysis.

### `Simulation.gas.Hp`

Pressure scale height of the gas given by the ratio of the isothermal sound speed to the Keplerian frequency.

$H_\mathrm{P} = \frac{c_{\mathrm{s},\,\mathrm{iso}}}{\Omega_\mathrm{K}}$.

### `Simulation.gas.mfp`

Mean free path of the gas in the midplane of the disk

$\lambda_\mathrm{mfp} = \frac{1}{\sqrt{2}\,n\,\sigma_\mathrm{H_2}}$

### `Simulation.gas.mu`

Mean molecular weight of the gas. This is initially set by the value given in `Simulation.ini.gas.mu` and is equal to $2.3\,m_\mathrm{P}$.

In [ ]:
sim.gas.mu

### `Simulation.gas.n`

Midplane number density of the gas given by

$n = \frac{\rho}{\mu}$

### `Simulation.gas.nu`

Kinematic viscosity of the gas given by

$\nu = \alpha c_\mathrm{s} H_\mathrm{P}$.

### `Simulation.gas.P`

Midplane gas pressure given by

$P = \frac{\rho\,c_\mathrm{s}^2}{\gamma}$.

### `Simulation.gas.rho`

Midplane gas mass density given by

$\rho = \frac{\Sigma_\mathrm{g}}{\sqrt{2\pi}H_\mathrm{P}}$.

### `Simulation.gas.Sigma`

Gas surface density. This is the quantity that is integrated with an implicit Euler first-order scheme.

### `Simulation.gas.SigmaFloor`

This is the floor value of the gas surface density. By default it is $10^{-100}$.

In [ ]:
sim.gas.SigmaFloor

If the gas surface density is at any point below it's floor value it will be automatically set to the floor value at the end of a time step.

### `Simulation.gas.T`

This is the midplane gas temperature. It is calculated by assuming a passively irradiated disk with a constant irradiation angle of $0.05$.

$T\left( r \right) = \sqrt[4]{\frac{0.05\,L_*}{4\,\pi\,r^2\,\sigma_\mathrm{SB}}}$

### Update order

The update order of the gas quantities in the standard model is set to

In [ ]:
sim.gas.updateorder

In [ ]:
sim.gas.S.updateorder

In [ ]:
sim.gas.v.updateorder

**Note**: The quantities that are excluded are calculated in the finalization step of the implicit integrator from the new values of the surface densities.

## Components

since we allow for multiple components to make up the gas disk i.e. different chemcial elements, we have a components group that gets initalized with one default component that makes up the entire gas in the disk

In [ ]:
sim.components

This default component alsways has a gas and dust group(which is empty for this default component)

In [ ]:
sim.components.Default

each component has its gas and dust contents. For the Default only gas is active (further functionality of components will be explaine on the next page)

In [ ]:
sim.components.Default.gas

### `Simulation.components.Default.gas.Sigma`

The surface desity of this specific gas species that contributes to the total Simulation.gas.Sigma

In [ ]:
all(sim.components.Default.gas.Sigma == sim.gas.Sigma)

### `Simulation.components.Default.gas.Sigma_dot`

The extenal source term of Simulation.components.Default.gas.Sigma

In [ ]:
sim.components.Default.gas.Sigma_dot

### `Simulation.components.Default.gas.Fi`

the radial flux of this component at the cell interfaces

In [ ]:
sim.components.Default.gas.Fi

### `Simulation.components.Default.gas.value`

if the component is set as a tracer tracking a property (e.g. gas that originated from a specific location), the value tracks the distribution of the tracer (e.g. the local density of gas in the disc from one original location after time) and value_dot is its external source term.

In [ ]:
sim.components.Default.gas.value

### `Simulation.components.Default.S`

Group containing the source terms of this components that are used for the time step calculation. The fields of the source terms for each components have the same structure and function as the source terms covered in `Simulation.gas.S` for the total gas.



In [ ]:
sim.components.Default.gas.S

### `Simulation.components.Default.pars.mu`

the mean molecular weight of this gas component 

In [ ]:
sim.components.Default.gas.pars.mu

## Grid

In [ ]:
sim.grid

These are all quantities that define the radial and the mass grid. Once they are defined they are constant and should not be changed. Additionally, the Keplerian frequency is located here.

### `Simulation.grid.A`

Surface area of the annulus a grid cell spans.

$A_i = \pi \left( r_{i+\frac{1}{2}}^2 - r_{i-\frac{1}{2}}^2 \right)$

### `Simulation.grid.Nr`

Number of radial grid cells.

In [ ]:
sim.grid.Nr

### `Simulation.grid.OmegaK`

Keplerian frequency given by

$\Omega_\mathrm{K} = \sqrt{\frac{G\,M_*}{r^3}}$.

### `Simulation.grid.r`

Radial grid cell centers. The radial grid cell centers are exactly in the middle between the grid cell interfaces.

In [ ]:
sim.grid.r == 0.5 * (sim.grid.ri[:-1] + sim.grid.ri[1:])

### `Simulation.grid.ri`

Locations of the grid cell interfaces. `Simulation.grid.ri[0]` and `Simulation.grid.ri[-1]` are the inner and outer grid boundaries.

### Update order

The update order of the grid quantities in the standard model is set to

In [ ]:
sim.grid.updateorder

## Star

In [ ]:
sim.star

### `Simulation.star.L`

Stellar luminosity given by

$L = 4\pi\,R_*^2\,\sigma_\mathrm{SB}\,T_*^4$

### `Simulation.star.M`

Stellar mass. Initially set to the value given in `Simulation.ini.star.M`, which corresponds to one Solar mass.

In [ ]:
sim.star.M

### `Simulation.star.R`

Stellar radius. Initially set to the value given in `Simulation.ini.star.R`, which corresponds to 2 Solar radii.

In [ ]:
sim.star.R

### `Simulation.star.T`

Stellar effective surface temperature. Initially set to the value given in `Simulation.ini.star.T`.

In [ ]:
sim.star.T

### Update order

The update order of the stellar quantities in the standard model is set to

In [ ]:
sim.star.updateorder

## Time

`Simulation.t` is the current time and the integration variable. It starts at zero initially.

In [ ]:
sim.t

Snapshots are written between $10^3$ years and $10^5$ years with 10 snapshots per time decade.

In [ ]:
sim.t.snapshots

The timestep is calculated from the dust and the gas source terms and the current values of the surface densities of dust and gas, while using a safety factor of `0.1`, which can be accessed as an attribute of the integration variable.

In [ ]:
sim.t.cfl

## Integrator

The integrator that is used by default as two integration instructions. One for the gas and one for the dust.

In [ ]:
sim.integrator

In [ ]:
sim.integrator.instructions

The gas is integrated with an implicit first-order Euler integration scheme. The Jacobian is calculated from the parameters given in `Simulation.gas`. Parameters like gas velocities, fluxes and source terms are calculated once the new values of the gas surface density have been found.

Except for the values at the boundaries, the advective source terms are given by

In [ ]:
sim.gas.Sigma.jacobian() @ sim.gas.Sigma

The total source terms, i.e., including external sources and excluding the boundaries are given by

In [ ]:
sim.gas.Sigma.jacobian() @ sim.gas.Sigma + sim.gas.S.ext

The dust is integrated with an implicit first-order Euler integration scheme. The Jacobian is calculated from the parameters given in `Simulation.dust`. Parameters like gas velocities, fluxes and source terms are calculated once the new values of the gas surface density have been found.

Except for the values at the boundaries, the hydrodynamic and coagulation source terms are given by

In [ ]:
(sim.dust.Sigma.jacobian() @ sim.dust.Sigma.ravel()).reshape(sim.dust.Sigma.shape)

Note the `ravel()` and `reshape()` operations, since the Jacobian works on a one-dimensional state vector.

The total source terms except at the boundaries are given by

In [ ]:
(sim.dust.Sigma.jacobian() @ sim.dust.Sigma.ravel()).reshape(sim.dust.Sigma.shape) + sim.dust.S.ext

At the end of a successful integration step, the floor values and boundaries are enforced.

the dutst integrator works much in the same way using a first oreder implicit scheme to update $\Sigma_0$, $\Sigma_1$ and $s_{max}$ at the same time:

In [ ]:
sim.dust.Sigma.jacobian() @ sim.dust.Sigma.ravel() + sim.dust.S.ext.ravel()

Note that growth term of $s_{max}$ has to be treated explicitly since it is highly non linear in the integation variables

## Writer

`TriPoD` uses by default the `hdf5writer` of `simframe`.

In [ ]:
sim.writer